# SCGNN Train

In [1]:
import pandas as pd
import torch
import sys
sys.path.append("..")
from src.config import PROJECT_ROOT
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader, random_split


In [2]:
df = pd.read_parquet(PROJECT_ROOT.joinpath('data/interim/sample.parquet'))
df = df.fillna(0)
# Küçük bir sensör alt kümesi kullan
df = df.iloc[:, :10]
X_STEP, Y_STEP = 2, 1
data_array = df.values


In [3]:
dataset = TrafficDataset(data_array, X_STEP, Y_STEP)
train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len
train_ds, val_ds = random_split(dataset, [train_len, val_len])
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=2)


In [4]:
sensors = data_array.shape[1]
edges = []
for i in range(sensors - 1):
    edges.append([i, i + 1])
    edges.append([i + 1, i])
edge_index = torch.tensor(edges, dtype=torch.long).t()
predictor = TrafficPredictor('scgnn', sensors, X_STEP, Y_STEP, edge_index=edge_index)
predictor.train(train_loader, val_loader, epochs=1)
predictor.show()


Epoch 1/1  Train Loss: 3313.1646  Val Loss: 1485.0063  Val Acc: 0.0000
== Son Performans Metrikleri ==
mse: 1485.00634765625
accuracy: 0.0
precision: 0.0
recall: 0.0
f1_score: 0.0
confusion_matrix: [[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [2 1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]]
